In [1]:
import os
from dotenv import load_dotenv
import requests
import openai

In [2]:
load_dotenv()

True

In [4]:
import requests
import os

FINNHUB_API_KEY = os.getenv("FINNHUB")
BASE_URL = "https://finnhub.io/api/v1/"

headers = {
    "X-Finnhub-Token": FINNHUB_API_KEY
}

def safe_extract(data, key):
    """Safely extract a value from a dictionary. If the key doesn't exist, return None."""
    return data.get(key, None)

def get_ratios_for_ticker(ticker):
    """Fetch various financial metrics for the given ticker."""
    response = requests.get(f"{BASE_URL}stock/metric?symbol={ticker}&metric=all", headers=headers)
    data = response.json()
    
    metrics = data.get('metric', {})
    
    pe_ratio = safe_extract(metrics, 'peNormalizedAnnual')
    pb_ratio = safe_extract(metrics, 'pbAnnual')
    ps_ratio = safe_extract(metrics, 'psAnnual')
    dividend_yield = safe_extract(metrics, 'dividendYieldIndicatedAnnual')
    roe = safe_extract(metrics, 'roeTTM')
    roa = safe_extract(metrics, 'roaTTM')
    debt_to_equity = safe_extract(metrics, 'totalDebt/totalEquityAnnual')
    current_ratio = safe_extract(metrics, 'currentRatioAnnual')
    quick_ratio = safe_extract(metrics, 'quickRatioAnnual')
    operating_margin = safe_extract(metrics, 'operatingMarginAnnual')
    gross_margin = safe_extract(metrics, 'grossMarginAnnual')
    pcash_flow = safe_extract(metrics, 'pcfShareAnnual')

    return pe_ratio, pb_ratio, ps_ratio, dividend_yield, roe, roa, debt_to_equity, current_ratio, quick_ratio, operating_margin, gross_margin, pcash_flow

def get_peers_of_ticker(ticker):
    """Fetch the list of peers for the given ticker."""
    response = requests.get(f"{BASE_URL}stock/peers?symbol={ticker}", headers=headers)
    peers = response.json()
    return peers

def get_peer_ratios(ticker):
    """Fetch average financial metrics for the peers of the given ticker."""
    peers = get_peers_of_ticker(ticker)
    metrics = {
        'pe_ratios': [], 'pb_ratios': [], 'ps_ratios': [], 'dividend_yields': [], 'roes': [],
        'roas': [], 'debt_to_equities': [], 'current_ratios': [], 'quick_ratios': [],
        'operating_margins': [], 'gross_margins': [], 'pcash_flows': []
    }

    for peer in peers:
        try:
            pe, pb, ps, dividend_yield, roe, roa, debt_to_equity, current_ratio, quick_ratio, operating_margin, gross_margin, pcash_flow = get_ratios_for_ticker(peer)
            if pe is not None: metrics['pe_ratios'].append(pe)
            if pb is not None: metrics['pb_ratios'].append(pb)
            if ps is not None: metrics['ps_ratios'].append(ps)
            if dividend_yield is not None: metrics['dividend_yields'].append(dividend_yield)
            if roe is not None: metrics['roes'].append(roe)
            if roa is not None: metrics['roas'].append(roa)
            if debt_to_equity is not None: metrics['debt_to_equities'].append(debt_to_equity)
            if current_ratio is not None: metrics['current_ratios'].append(current_ratio)
            if quick_ratio is not None: metrics['quick_ratios'].append(quick_ratio)
            if operating_margin is not None: metrics['operating_margins'].append(operating_margin)
            if gross_margin is not None: metrics['gross_margins'].append(gross_margin)
            if pcash_flow is not None: metrics['pcash_flows'].append(pcash_flow)
        except:
            pass

    avg_metrics = {key: sum(values) / len(values) if values else None for key, values in metrics.items()}
    return avg_metrics

def main():
    ticker = input("Enter the ticker symbol: ").upper()
    
    # Fetch ratios for the given ticker
    pe, pb, ps, dividend_yield, roe, roa, debt_to_equity, current_ratio, quick_ratio, operating_margin, gross_margin, pcash_flow = get_ratios_for_ticker(ticker)
    print(f"\n{ticker} Metrics:")
    print(f"P/E Ratio: {pe}")
    print(f"P/B Ratio: {pb}")
    print(f"P/S Ratio: {ps}")
    print(f"Dividend Yield: {dividend_yield}")
    print(f"ROE: {roe}")
    print(f"ROA: {roa}")
    print(f"Debt-to-Equity Ratio: {debt_to_equity}")
    print(f"Current Ratio: {current_ratio}")
    print(f"Quick Ratio: {quick_ratio}")
    print(f"Operating Margin: {operating_margin}")
    print(f"Gross Margin: {gross_margin}")
    print(f"Price-to-Cash Flow: {pcash_flow}")

    # Fetch average ratios for the peers of the given ticker
    avg_metrics = get_peer_ratios(ticker)
    print("\nAverage Metrics Among Peers:")
    # Create a mapping for metric names
    metric_names = {
        'pe_ratios': 'P/E Ratio',
        'pb_ratios': 'P/B Ratio',
        'ps_ratios': 'P/S Ratio',
        'dividend_yields': 'Dividend Yield',
        'roes': 'ROE',
        'roas': 'ROA',
        'debt_to_equities': 'Debt-to-Equity Ratio',
        'current_ratios': 'Current Ratio',
        'quick_ratios': 'Quick Ratio',
        'operating_margins': 'Operating Margin',
        'gross_margins': 'Gross Margin',
        'pcash_flows': 'Price-to-Cash Flow'
    }
    for key, value in avg_metrics.items():
        print(f"{metric_names[key]}: {value}")

if __name__ == "__main__":
    main()


Enter the ticker symbol:  PTON



PTON Metrics:
P/E Ratio: None
P/B Ratio: 5.2376
P/S Ratio: 0.5948
Dividend Yield: None
ROE: -418.40999999999997
ROA: -39.800000000000004
Debt-to-Equity Ratio: 2.6337
Current Ratio: 2.1527
Quick Ratio: 1.1966
Operating Margin: -42.75
Gross Margin: 32.98
Price-to-Cash Flow: None

Average Metrics Among Peers:
P/E Ratio: 18.485766666666667
P/B Ratio: 3.1387363636363643
P/S Ratio: 1.0627454545454547
Dividend Yield: 3.0224385999999996
ROE: -24.234545454545454
ROA: 2.5009090909090905
Debt-to-Equity Ratio: 0.8589454545454547
Current Ratio: 1.918618181818182
Quick Ratio: 0.8779454545454545
Operating Margin: 5.057272727272728
Gross Margin: 38.75727272727272
Price-to-Cash Flow: 16.755924999999998


In [9]:



# Initialize the OpenAI API with your key
openai.api_key = os.getenv("AI_KEY")  # Replace with your actual key

def analyze_ratios_with_openai(ticker, metrics, avg_metrics):
    # Constructing a prompt for the OpenAI API
    prompt = f"""
    Analyze the financial metrics for the company with ticker symbol {ticker}:

    - P/E Ratio (Price-to-Earnings Ratio) for {ticker}: {metrics['pe']}
      Average P/E Ratio among peers: {avg_metrics['pe_ratios']}

    - P/B Ratio (Price-to-Book Ratio) for {ticker}: {metrics['pb']}
      Average P/B Ratio among peers: {avg_metrics['pb_ratios']}

    - P/S Ratio (Price-to-Sales Ratio) for {ticker}: {metrics['ps']}
      Average P/S Ratio among peers: {avg_metrics['ps_ratios']}

    - Dividend Yield for {ticker}: {metrics['dividend_yield']}
      Average Dividend Yield among peers: {avg_metrics['dividend_yields']}

    - ROE (Return on Equity) for {ticker}: {metrics['roe']}
      Average ROE among peers: {avg_metrics['roes']}

    - ROA (Return on Assets) for {ticker}: {metrics['roa']}
      Average ROA among peers: {avg_metrics['roas']}

    - Debt-to-Equity Ratio for {ticker}: {metrics['debt_to_equity']}
      Average Debt-to-Equity Ratio among peers: {avg_metrics['debt_to_equities']}

    - Current Ratio for {ticker}: {metrics['current_ratio']}
      Average Current Ratio among peers: {avg_metrics['current_ratios']}

    - Quick Ratio for {ticker}: {metrics['quick_ratio']}
      Average Quick Ratio among peers: {avg_metrics['quick_ratios']}

    - Operating Margin for {ticker}: {metrics['operating_margin']}
      Average Operating Margin among peers: {avg_metrics['operating_margins']}

    - Gross Margin for {ticker}: {metrics['gross_margin']}
      Average Gross Margin among peers: {avg_metrics['gross_margins']}

    - Price-to-Cash Flow for {ticker}: {metrics['pcash_flow']}
      Average Price-to-Cash Flow among peers: {avg_metrics['pcash_flows']}

   Considering the above data points and understanding the importance of these metrics in evaluating a company's financial health and performance:

1. How does the company stand in terms of profitability, liquidity, and solvency compared to its peers?
2. Are there any alarming disparities or noteworthy strengths in any specific metrics?
3. What might these metrics indicate about the company's operational efficiency, financial strategies, or market positioning?
4. Are there potential opportunities or risks that these metrics highlight?

Please provide a comprehensive analysis of the company's financial standing compared to its peers.

    """

    # Querying the OpenAI API
    response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, max_tokens=400)
    analysis = response.choices[0].text.strip()

    return analysis

ticker = "PTON"
pe, pb, ps, dividend_yield, roe, roa, debt_to_equity, current_ratio, quick_ratio, operating_margin, gross_margin, pcash_flow = get_ratios_for_ticker(ticker)

metrics = {
    'pe': pe,
    'pb': pb,
    'ps': ps,
    'dividend_yield': dividend_yield,
    'roe': roe,
    'roa': roa,
    'debt_to_equity': debt_to_equity,
    'current_ratio': current_ratio,
    'quick_ratio': quick_ratio,
    'operating_margin': operating_margin,
    'gross_margin': gross_margin,
    'pcash_flow': pcash_flow
}

avg_metrics = get_peer_ratios(ticker)

analysis_result = analyze_ratios_with_openai(ticker, metrics, avg_metrics)
print(analysis_result)


Based on the financial metrics provided, it appears that PTON is not as profitable, liquid, or solvent as its peers. In terms of profitability, PTON has an operating margin of -42.75% and a gross margin of 32.98%, both of which are significantly lower than the average operating margin and gross margin of 5.06% and 38.76% among its peers, respectively. This indicates that PTON is not as efficient in its operations as its peers and is not generating as much profit from its sales. In terms of liquidity, PTON has a current ratio and quick ratio of 2.15 and 1.20, respectively, which are both lower than the average current ratio and quick ratio of 1.92 and 0.88 among its peers. This indicates that PTON has less cash and assets on hand to cover its short-term obligations than its peers. In terms of solvency, PTON has a debt-to-equity ratio of 2.63, which is significantly higher than the average debt-to-equity ratio of 0.86 among its peers. This indicates that PTON has a higher amount of debt 